In [2]:
def print_dict(d):
    for k, v in d.items():
        print(f"\t'{k}': {v}")

def print_dict2(d):
    for detail in d:
        print()

        i = 0
        for k, v in detail.items():
            if i == 0:
                print(f"\t'{k}': {v}")
            else:
                print(f"\t\t'{k}': {v}")

In [3]:
import os
import platform
import sys

if platform.system() == 'Windows':
    midi_ddsp_module_path = os.path.abspath(os.path.join('../../'))
    ddsp_module_path = os.path.abspath(os.path.join('../../../ddsp-playground-2/'))
else:
    midi_ddsp_module_path = os.path.abspath(os.path.join('../../'))
    ddsp_module_path = os.path.abspath(os.path.join('../../../ddsp/ddsp-playground-2/'))

def apply_module_path(module_path):
    print(f"module_path={module_path}")
    if module_path not in sys.path:
      sys.path.append(module_path)
      print(f"appending {module_path} to sys.path")
    else:
      print(f"do not appending {module_path} to sys.path")

apply_module_path(midi_ddsp_module_path)
apply_module_path(ddsp_module_path)

import sys
if platform.system() != 'Windows':
    sparsenet_module_path_abs = '/ssd003/home/burakovr/projects/vova/envs/main/lib/python3.8/site-packages/'
    apply_module_path(sparsenet_module_path_abs)

import midi_ddsp

module_path=E:\Code\Projects\TimbreTransfer\midi-ddsp-main
do not appending E:\Code\Projects\TimbreTransfer\midi-ddsp-main to sys.path
module_path=E:\Code\Projects\TimbreTransfer\ddsp-playground-2
appending E:\Code\Projects\TimbreTransfer\ddsp-playground-2 to sys.path


E:\Code\Projects\TimbreTransfer\midi-ddsp-main\midi_ddsp\midi_ddsp_synthesize.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


nn.py sys.path=['E:\\Code\\Software\\PyCharm 2021.3.2\\plugins\\python\\helpers-pro\\jupyter_debug', 'E:\\Code\\Software\\PyCharm 2021.3.2\\plugins\\python\\helpers\\pydev', 'E:\\Code\\Projects\\TimbreTransfer\\midi-ddsp-main\\midi_ddsp\\colab', 'E:\\Code\\Projects\\TimbreTransfer\\midi-ddsp-main', 'E:\\Code\\Software\\Python38\\python38.zip', 'E:\\Code\\Software\\Python38\\DLLs', 'E:\\Code\\Software\\Python38\\lib', 'E:\\Code\\Software\\Python38', 'e:\\code\\projects\\coursework\\ddsp-playground\\ddsp-playground-2', '', 'e:\\code\\projects\\coursework\\ddsp-playground\\ddsp-playground-2\\lib\\site-packages', 'e:\\code\\projects\\coursework\\ddsp-playground\\ddsp-playground-2\\lib\\site-packages\\win32', 'e:\\code\\projects\\coursework\\ddsp-playground\\ddsp-playground-2\\lib\\site-packages\\win32\\lib', 'e:\\code\\projects\\coursework\\ddsp-playground\\ddsp-playground-2\\lib\\site-packages\\Pythonwin', 'E:\\Code\\Projects\\TimbreTransfer\\ddsp-playground-2', 'E:/Code/Projects/Coursewo

In [4]:
#  Copyright 2022 The MIDI-DDSP Authors.
#  #
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#  #
#      http://www.apache.org/licenses/LICENSE-2.0
#  #
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

"""Training code for Synthesis Generator."""

import tensorflow as tf
import time
import os
import sys
import logging
import argparse
import IPython

from keras.utils.layer_utils import print_summary
from livelossplot import PlotLosses

from midi_ddsp.data_handling.get_dataset import get_dataset
from midi_ddsp.utils.training_utils import print_hparams, set_seed, \
    save_results, str2bool
from midi_ddsp.utils.summary_utils import write_tensorboard_audio
#                          from midi_ddsp.hparams_synthesis_generator import hparams as hp
from midi_ddsp.hparams_synthesis_generator import hparams_debug as hp
from midi_ddsp.modules.recon_loss import ReconLossHelper
from midi_ddsp.modules.gan_loss import GANLossHelper
from midi_ddsp.modules.get_synthesis_generator import get_synthesis_generator, \
    get_fake_data_synthesis_generator
from midi_ddsp.modules.discriminator import Discriminator
from ddsp.colab.notebook_utils import play, specplot


In [5]:
print(tf.__version__)

2.9.3


In [6]:
model_path = "DDSP_VST_Trained_model/"

In [7]:
#import ddsp
#from ddsp.training import get_inference_model
model = tf.saved_model.load(model_path)

KeyboardInterrupt: 

In [17]:
# Get the model's input signature
print(model.signatures.keys())

input_signature = model.signatures['call']
print(input_signature)

# Get the input shape
input_shape = input_signature.structured_input_signature[1]['input_1'].shape
print("Input shape:", input_shape)

KeysView(_SignatureMap({'call': <ConcreteFunction signature_wrapper(*, state, pw_scaled, f0_scaled) at 0x20AE34046D0>}))
ConcreteFunction signature_wrapper(*, state, pw_scaled, f0_scaled)
  Args:
    f0_scaled: float32 Tensor, shape=(1,)
    pw_scaled: float32 Tensor, shape=(1,)
    state: float32 Tensor, shape=(512,)
  Returns:
    {'output_0': <1>, 'output_1': <2>, 'output_2': <3>, 'output_3': <4>}
      <1>: float32 Tensor, shape=(1,)
      <2>: float32 Tensor, shape=(60,)
      <3>: float32 Tensor, shape=(65,)
      <4>: float32 Tensor, shape=(512,)


KeyError: 'input_1'

In [14]:
model = tf.saved_model.load(model_path)
print(model)
print(model.layers[0])
print(model.layers[0].input_shape)
#print(tf.keras.Model.summary(model, expand_nested=True))

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x0000020AE8058610>


AttributeError: '_UserObject' object has no attribute 'layers'

In [18]:
import tensorflow as tf

# Replace with your TFRecord file path
tfrecord_path = 'C:/Users/vovab/Downloads/train.tfrecord-00001-of-00010'

# Create a TFRecordDataset to read the records
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Function to parse a single example
def parse_example(example_proto):
    # Define the feature specification for your dataset
    feature_spec = {
        'field1': tf.io.FixedLenFeature([], tf.string),
        'field2': tf.io.FixedLenFeature([], tf.int64),
        # ... add other fields as needed
    }

    # Parse the example
    parsed_example = tf.io.parse_single_example(example_proto, feature_spec)

    return parsed_example

# Iterate through the records and print the structure of the first record
for raw_record in dataset.take(1):
    example = parse_example(raw_record)
    print("Fields and shapes:")
    for field, value in example.items():
        print(f"{field}: {value.shape}")

InvalidArgumentError: Feature: field1 (data type: string) is required but could not be found. [Op:ParseExampleV2]